
# 22p22e0114_กวี
# W2HW2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os

In [ ]:
cd ../input/titanic-machine-learning-from-disaster

# โหลดข้อมูลเข้า kaggle

In [ ]:
training = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
training.head()

In [ ]:
training.info()

# Data cleaning ทำการ Clean ข้อมูลก่อนทำการส่งเข้าเทรนโมเดลเป็นลำดับถัดไป

ตรวจสอบข้อมูลที่ขาดหายไป (NaN)

In [ ]:
def null_data(training, test):
    print("Missing on training")
    print(pd.isnull(training).sum()) 
    print("\n","Missing on test")
    print(pd.isnull(test).sum())

null_data(training, test)

## จากการวิเคราะห์ข้อมูล ทำการ Drop column ที่ไม่ใช้ออกไป
1. ticket มีข้อมูลตัวเลขที่ค่อนข้างซับซ้อน ไม่ควรนำมาใช้ต่อ
2. cabin มี Missing value เยอะเกินไป ไม่ควรใช้ต่อ

In [ ]:
training.drop(labels = ["Cabin", "Ticket"], axis = 1, inplace = True)
test.drop(labels = ["Cabin", "Ticket"], axis = 1, inplace = True)

null_data(training, test)

# เติม Missing value

In [ ]:
# เติมข้อมูลที่เป็น Na ด้วยค่า median

training["Age"].fillna(training["Age"].median(), inplace = True)
test["Age"].fillna(test["Age"].median(), inplace = True) 
training["Embarked"].fillna("S", inplace = True)
test["Fare"].fillna(test["Fare"].median(), inplace = True)

null_data(training, test)

> # ทำ One hot encoding บนข้อมูลที่เป็น Categorical

สร้างฟังก์ชันสำหรับทำ One hot encoding บน Sex และ Embarked

In [ ]:
def transform_one_hot_encoding_for_titanic(df):
    dummies_sex = pd.get_dummies(df["Sex"],drop_first = True) # Drop_First กำหนดเพื่อป้องกันปัญหา multicollinearity
    dummies_embarked = pd.get_dummies(df["Embarked"],drop_first = True)
    merged = pd.concat([df,dummies_sex,dummies_embarked],axis='columns')
    merged = merged.drop(["Sex","Embarked","Name","PassengerId"],axis=1) # Drop unused columns
    merged = merged.rename(columns={"male":"is_male"})
    return merged
    

In [ ]:
pre_training = transform_one_hot_encoding_for_titanic(training)
pre_test = transform_one_hot_encoding_for_titanic(test)

In [ ]:
pre_test.head()


In [ ]:
pre_training.head()

# เขียนฟังก์ชันสำหรับ Normalize ข้อมูลใน Column ที่ต้องการให้อยู่ในรูป Z-score

In [ ]:
import pandas as pd
from sklearn import preprocessing

def normalize_z_column(df,cols_list): # ฟังก์ชันสำหรับ normalize df columns ที่ต้องการ
    pre_process = preprocessing.PowerTransformer(method="yeo-johnson",standardize = True)
    temp1  = pre_process.fit_transform(df[cols_list])
    temp2 = pd.DataFrame(temp1,columns = cols_list)
    pre_merge = df.drop(cols_list,axis=1)
    merge = pd.concat([pre_merge,temp2],axis=1)


    return merge


In [ ]:
cols = ['Fare','Age'] # Cols that we need to normalize

final_training = normalize_z_column(pre_training,cols)
final_test = normalize_z_column(pre_test,cols)

ได้ผลลัพธ์สุดท้ายก่อนทำการ Feed เข้าโมเดล

In [ ]:
final_training.head()

# เริ่มเขียน Code สำหรับการ Train โมเดล

แบ่งชุดข้อมูลเป็น 5-Fold Cross validation

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.get_n_splits(final_training)

In [ ]:
print(kf)

- สร้าง Array เก็บค่า X และ y ที่ต้องการเทรนออกมาจาก DataFrame

In [ ]:
X = final_training[["Pclass","SibSp","Parch","is_male","Q","S","Fare","Age"]].to_numpy()
y = final_training[["Survived"]].to_numpy()

# กำหนด model Decision tree และ Naive bayes สำหรับการทำ Classification 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, accuracy_score 
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support # สำหรับคำนวณ Precision Recall F1 Score

# กำหนด MODEL ต่างๆที่จะใช้งาน
# โมเดล Decision Tree และ Naive Bayes
model_t = DecisionTreeClassifier()
model_nb = GaussianNB()

score_tree = []
recall_tree_class0 = []
precision_tree_class0 = []
F_score_tree_class0 = []
recall_tree_class1 = []
precision_tree_class1 = []
F_score_tree_class1 = []


score_naive = []
recall_naive_class0 = []
precision_naive_class0 = []
F_score_naive_class0 = []
recall_naive_class1 = []
precision_naive_class1 = []
F_score_naive_class1 = []



i = 0
print("START TRAINING ON TREE MODEL")
for train_index, validation_index in kf.split(X):
    i += 1
    print("Fold number",i)
   
    X_train,X_val = X[train_index], X[validation_index]
    y_train,y_val = y[train_index], y[validation_index]

    model_t.fit(X_train,y_train.reshape(y_train.shape[0]))
    pred = model_t.predict(X_val)
    acc_dt = accuracy_score(y_val,pred)
    score_tree.append(acc_dt)
    print("score  is:",acc_dt)
    
    # Calculate Precision,Recall,F1score
    temp_score = precision_recall_fscore_support(y_val, pred,average=None)
    
    precision_tree_class0.append(temp_score[0][0])
    recall_tree_class0.append(temp_score[1][0])
    F_score_tree_class0.append(temp_score[2][0])
    
    
    precision_tree_class1.append(temp_score[0][1])
    recall_tree_class1.append(temp_score[1][1])
    F_score_tree_class1.append(temp_score[2][1])
print("END OF TRAINING ON TREE MODEL","\n")

print("Score tree mean is:",np.array(score_tree).mean(),"\n")

print("START TRAINING ON NAIVE BAYES MODEL")    
i = 0  
for train_index, validation_index in kf.split(X):
    i += 1
    print("Fold number",i)
    X_train,X_val = X[train_index], X[validation_index]
    y_train,y_val = y[train_index], y[validation_index]

    model_nb.fit(X_train,y_train.reshape(y_train.shape[0]))
    pred = model_nb.predict(X_val)
    acc_dt = accuracy_score(y_val,pred)
    score_naive.append(acc_dt)
    print("score  is:",acc_dt)
    
    # Calculate Precision,Recall,F1score
    temp_score = precision_recall_fscore_support(y_val, pred,average=None)
    
    precision_naive_class0.append(temp_score[0][0])
    recall_naive_class0.append(temp_score[1][0])
    F_score_naive_class0.append(temp_score[2][0])
    
    
    precision_naive_class1.append(temp_score[0][1])
    recall_naive_class1.append(temp_score[1][1])
    F_score_naive_class1.append(temp_score[2][1])
print("END OF TRAINING ON NAIVE BAYES","\n")

print("Score naive bayes is:",np.array(score_naive).mean())

    

# กำหนด model deep neural network สำหรับการทำ Classification 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense


# โมเดล Deep_neural_network
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss="binary_crossentropy",optimizer='adam',metrics=["accuracy"])
    return model

model_nn = create_model()
score_nn = []


recall_nn_class0 = []
precision_nn_class0 = []
F_score_nn_class0 = []
recall_nn_class1 = []
precision_nn_class1 = []
F_score_nn_class1 = []



i = 0
for train_index, validation_index in kf.split(X):
    i += 1
    print("Fold number",i)
    X_train,X_val = X[train_index], X[validation_index]
    y_train,y_val = y[train_index], y[validation_index]

    model_nn.fit(X_train,y_train,epochs=100,batch_size=10)
    pred = model_nn.predict(X_val)
    threshold = 0.5
    pred = pred >0.5 # element wise check to binary
    acc_dt = accuracy_score(y_val,pred)
    score_nn.append(acc_dt)
    print("score  is:",acc_dt)
    print("DONE")
    
    # Calculate Precision,Recall,F1score
    temp_score = precision_recall_fscore_support(y_val, pred,average=None)
    
    precision_nn_class0.append(temp_score[0][0])
    recall_nn_class0.append(temp_score[1][0])
    F_score_nn_class0.append(temp_score[2][0])
    
    
    precision_nn_class1.append(temp_score[0][1])
    recall_nn_class1.append(temp_score[1][1])
    F_score_nn_class1.append(temp_score[2][1])

    



## คำนวนผลลัพธ์เฉลี่ยจากข้อมูลที่ได้จากการทดลองจำนวน 5Folds

In [ ]:
# แสดงผลลัพท์การจำแนกในรูปแบบ 


# Recall ของแต่ละ Class

recall_mean_tree_class0 = np.array(recall_tree_class0).mean()
recall_mean_tree_class1 = np.array(recall_tree_class1).mean()
recall_mean_naive_class0 = np.array(recall_naive_class0).mean()
recall_mean_naive_class1 = np.array(recall_naive_class1).mean()
recall_mean_nn_class0 = np.array(recall_nn_class0).mean()
recall_mean_nn_class1 = np.array(recall_nn_class1).mean()


# Precision ของแต่ละ Class
precision_mean_tree_class0 = np.array(precision_tree_class0).mean()
precision_mean_tree_class1 = np.array(precision_tree_class1).mean()
precision_mean_naive_class0 = np.array(precision_naive_class0).mean()
precision_mean_naive_class1 = np.array(precision_naive_class1).mean()
precision_mean_nn_class0 = np.array(precision_nn_class0).mean()
precision_mean_nn_class1 = np.array(precision_nn_class1).mean()




# F-Measure ของแต่ละ Class
F_score_mean_tree_class0 = np.array(F_score_tree_class0).mean()
F_score_mean_tree_class1 = np.array(F_score_tree_class1).mean()
F_score_mean_naive_class0 = np.array(F_score_naive_class0).mean()
F_score_mean_naive_class1 = np.array(F_score_naive_class1).mean()
F_score_mean_nn_class0 = np.array(F_score_nn_class0).mean()
F_score_mean_nn_class1 = np.array(F_score_nn_class1).mean()



# Average F-Measure ของทั้งชุดข้อมูล
FF_average_tree = np.array([F_score_mean_tree_class0,F_score_mean_tree_class1]).mean()
FF_average_naive = np.array([F_score_mean_naive_class0,F_score_mean_naive_class1]).mean()
FF_average_nn = np.array([F_score_mean_nn_class0,F_score_mean_nn_class1]).mean()


In [ ]:
def result_df(r0,r1,p0,p1,f0,f1):
    d ={"Recall":[r0,r1],"Precision":[p0,p1],"F-Score":[f0,f1]}
    df = pd.DataFrame(data=d)
    df.index.name="CLASS"
    return df

In [ ]:
tree_result = result_df(recall_mean_tree_class0,recall_mean_tree_class1,precision_mean_tree_class0,precision_mean_tree_class1,F_score_mean_tree_class0,F_score_mean_tree_class1)
nb_result = result_df(recall_mean_naive_class0,recall_mean_naive_class1,precision_mean_naive_class0,precision_mean_naive_class1,F_score_mean_naive_class0,F_score_mean_naive_class1)
nn_result = result_df(recall_mean_nn_class0,recall_mean_nn_class1,precision_mean_nn_class0,precision_mean_nn_class1,F_score_mean_nn_class0,F_score_mean_nn_class1)

In [ ]:
print("DECISION TREE MODEL")
print("Average F-Measure =", FF_average_tree)
print("Recall, Precision, F-score")
tree_result.head()

In [ ]:
print("NAIVE BAYES MODEL")
print("Average F-Measure =", FF_average_naive)
print("Recall, Precision, F-score")
nb_result.head()

In [ ]:
print("DEEP NEURAL NETWORK MODEL")
print("Average F-Measure =", FF_average_nn)
print("Recall, Precision, F-score")
nn_result.head()

จากโมเดล พบว่า Deep neural network ให้คะแนนภาพรวมที่ดีกว่า อย่างไรก็ตามเราสามารถปรับ Hyperparameter ของโมเดลต่างๆเพื่อให้ผลลัพธ์ดีขึ้นได้อีก
